In [1]:
!pip install sacrebleu bert-score torchmetrics nltk rouge-score datasets transformers groq pandas tqdm matplotlib seaborn flash_attn
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | / done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6686219aee8ca4f285604bb3011882c31699a385c7eef48f96c8dbcebaac37ac
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
  Created wheel for flash_attn: filename=flash_attn-2.7.0.post2-cp310-cp310-linux_x86_64.whl size=183279716 sha256=c1021a1c990422f49f554a42308c4dbfb14501d6c

True

In [2]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from sacrebleu.metrics import BLEU, CHRF, TER
from bert_score import BERTScorer
from torchmetrics.text import TranslationEditRate, WordErrorRate, CharErrorRate
from rouge_score import rouge_scorer
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import torch

warnings.filterwarnings('ignore')

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define models
model_names = ["microsoft/Phi-3.5-mini-instruct"]

def load_translation_data(language_pair, num_samples=20):
    """Load dataset for specified language pair."""
    try:
        dataset = load_dataset("wmt19", language_pair, split="validation")
    except ValueError:
        dataset = load_dataset("wmt19", language_pair, split="train")
    return dataset.select(range(min(num_samples, len(dataset))))

def translate_text(text, source_lang, target_lang, model_name, tokenizer, model):
    """Translate text using the specified model."""
    prompt = f"Translate the following text from {source_lang} to {target_lang}: {text}"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=256, truncation=True).to(device)
    
    # Generate translation with no gradients
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

def calculate_metrics(references, hypotheses):
    """Calculate various MT evaluation metrics."""
    bleu = BLEU()
    chrf = CHRF()
    ter_metric = TER()
    bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)
    wer = WordErrorRate()
    cer = CharErrorRate()
    rouge_metrics = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    bleu_score = bleu.corpus_score(hypotheses, [references]).score
    chrf_score = chrf.corpus_score(hypotheses, [references]).score
    ter_score = ter_metric.corpus_score(hypotheses, [references]).score
    P, R, F1 = bert_scorer.score(hypotheses, references)
    bert_score = F1.mean().item()
    wer_score = wer(hypotheses, references).item()
    cer_score = cer(hypotheses, references).item()
    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    
    for hyp, ref in zip(hypotheses, references):
        scores = rouge_metrics.score(ref, hyp)
        rouge_scores['rouge1'] += scores['rouge1'].fmeasure
        rouge_scores['rouge2'] += scores['rouge2'].fmeasure
        rouge_scores['rougeL'] += scores['rougeL'].fmeasure
    
    for key in rouge_scores:
        rouge_scores[key] /= len(hypotheses)

    return {
        "BLEU": bleu_score,
        "chrF": chrf_score,
        "TER": ter_score,
        "BERTScore": bert_score,
        "WER": wer_score,
        "CER": cer_score,
        "ROUGE-1": rouge_scores['rouge1'],
        "ROUGE-2": rouge_scores['rouge2'],
        "ROUGE-L": rouge_scores['rougeL']
    }

def plot_confusion_matrix(references, translations, model_name, pair_code):
    """Generate and save a confusion matrix."""
    matrix = np.zeros((len(references), len(translations)))
    for i, ref in enumerate(references):
        for j, hyp in enumerate(translations):
            matrix[i, j] = len(set(ref.split()) & set(hyp.split())) / len(set(ref.split()) | set(hyp.split()))
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(matrix, annot=False, cmap="coolwarm", cbar=True)
    plt.title(f"Confusion Matrix for {model_name} ({pair_code})")
    plt.xlabel("Hypotheses")
    plt.ylabel("References")
    plt.tight_layout()
    plt.savefig(f"confusion_matrix_{model_name}_{pair_code}.png")
    plt.close()

def evaluate_model(dataset, source_lang, target_lang, model_name):
    """Evaluate a specific model on the dataset."""
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)
    
    translations = []
    references = []
    source_texts = []
    
    for example in tqdm(dataset, desc=f"Translating with {model_name}"):
        source_text = example['translation'][source_lang]
        reference = example['translation'][target_lang]
        
        try:
            translation = translate_text(source_text, source_lang, target_lang, model_name, tokenizer, model)
            source_texts.append(source_text)
            translations.append(translation)
            references.append(reference)
        except Exception as e:
            print(f"Error during translation with {model_name}: {str(e)}")
            continue

    # Clear GPU memory after processing
    model.to("cpu")
    torch.cuda.empty_cache()

    if translations:
        metrics = calculate_metrics(references, translations)
        return source_texts, references, translations, metrics
    return [], [], [], {}

def main():
    language_pairs = [
        ("cs-en", "Czech-English"),
        ("de-en", "German-English"),
        ("fi-en", "Finnish-English"),
        ("fr-de", "French-German"),
        ("gu-en", "Gujarati-English"),
        ("kk-en", "Kazakh-English"),
        ("lt-en", "Lithuanian-English"),
        ("ru-en", "Russian-English"),
        ("zh-en", "Chinese-English")
    ]
    
    for pair_code, pair_name in language_pairs:
        print(f"\nEvaluating {pair_name} translations...")
        dataset = load_translation_data(pair_code, num_samples=50)
        source_lang, target_lang = pair_code.split("-")
        
        for model_name in model_names:
            print(f"Evaluating {model_name}...")
            source_texts, references, translations, metrics = evaluate_model(dataset, source_lang, target_lang, model_name)
            
            # Save metrics
            results_df = pd.DataFrame([metrics], index=[model_name.split("/")[-1]])
            results_df.to_csv(f"mt_evaluation_results_{model_name.split('/')[-1]}_{pair_code}.csv")
            
            # Save translations for review
            translations_df = pd.DataFrame({
                "Source": source_texts,
                "Reference": references,
                "Translation": translations
            })
            translations_df.to_csv(f"translations_{model_name.split('/')[-1]}_{pair_code}.csv", index=False)
            
            # Plot and save confusion matrix
            plot_confusion_matrix(references, translations, model_name.split("/")[-1], pair_code)
            
            # Print metrics
            print(f"\nResults for {model_name} ({pair_name}):")
            print(results_df)

if __name__ == "__main__":
    main()

Using device: cuda

Evaluating Czech-English translations...


README.md:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/216M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/3981 [00:00<?, ? examples/s]

Evaluating microsoft/Phi-3.5-mini-instruct...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Translating with microsoft/Phi-3.5-mini-instruct: 100%|██████████| 50/50 [08:58<00:00, 10.77s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Results for microsoft/Phi-3.5-mini-instruct (Chinese-English):
                           BLEU       chrF         TER  BERTScore       WER  \
Phi-3.5-mini-instruct  9.645664  43.208601  189.255189  -0.155542  1.944444   

                            CER   ROUGE-1  ROUGE-2   ROUGE-L  
Phi-3.5-mini-instruct  2.113674  0.399524   0.2107  0.336844  


In [3]:
# import numpy as np
# import pandas as pd
# from datasets import load_dataset
# from sacrebleu.metrics import BLEU, CHRF, TER
# from bert_score import BERTScorer
# from torchmetrics.text import TranslationEditRate, WordErrorRate, CharErrorRate
# from rouge_score import rouge_scorer
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from tqdm import tqdm
# import warnings
# import matplotlib.pyplot as plt
# import seaborn as sns
# import torch

# warnings.filterwarnings('ignore')

# # Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Define models
# model_names = ["microsoft/Phi-3.5-mini-instruct"]

# def load_translation_data(language_pair, num_samples=20):
#     """Load dataset for specified language pair."""
#     try:
#         dataset = load_dataset("wmt19", language_pair, split="validation")
#     except ValueError:
#         dataset = load_dataset("wmt19", language_pair, split="train")
#     return dataset.select(range(min(num_samples, len(dataset))))

# def translate_text(text, source_lang, target_lang, model_name, tokenizer, model):
#     """Translate text using the specified model."""
#     prompt = f"Translate the following text from {source_lang} to {target_lang}: {text}"
#     inputs = tokenizer(prompt, return_tensors="pt", max_length=256, truncation=True).to(device)
    
#     # Generate translation with no gradients
#     with torch.no_grad():
#         outputs = model.generate(**inputs, max_length=256)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# def calculate_metrics(references, hypotheses):
#     """Calculate various MT evaluation metrics."""
#     bleu = BLEU()
#     chrf = CHRF()
#     ter_metric = TER()
#     bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)
#     wer = WordErrorRate()
#     cer = CharErrorRate()
#     rouge_metrics = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
#     bleu_score = bleu.corpus_score(hypotheses, [references]).score
#     chrf_score = chrf.corpus_score(hypotheses, [references]).score
#     ter_score = ter_metric.corpus_score(hypotheses, [references]).score
#     P, R, F1 = bert_scorer.score(hypotheses, references)
#     bert_score = F1.mean().item()
#     wer_score = wer(hypotheses, references).item()
#     cer_score = cer(hypotheses, references).item()
#     rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    
#     for hyp, ref in zip(hypotheses, references):
#         scores = rouge_metrics.score(ref, hyp)
#         rouge_scores['rouge1'] += scores['rouge1'].fmeasure
#         rouge_scores['rouge2'] += scores['rouge2'].fmeasure
#         rouge_scores['rougeL'] += scores['rougeL'].fmeasure
    
#     for key in rouge_scores:
#         rouge_scores[key] /= len(hypotheses)

#     return {
#         "BLEU": bleu_score,
#         "chrF": chrf_score,
#         "TER": ter_score,
#         "BERTScore": bert_score,
#         "WER": wer_score,
#         "CER": cer_score,
#         "ROUGE-1": rouge_scores['rouge1'],
#         "ROUGE-2": rouge_scores['rouge2'],
#         "ROUGE-L": rouge_scores['rougeL']
#     }

# def evaluate_model(dataset, source_lang, target_lang, model_name):
#   """Evaluate a specific model on the dataset."""
#   tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#   model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)
    
#     translations = []
#     references = []
#     source_texts = []
    
#     for example in tqdm(dataset, desc=f"Translating with {model_name}"):
#         source_text = example['translation'][source_lang]
#         reference = example['translation'][target_lang]
        
#         try:
#             translation = translate_text(source_text, source_lang, target_lang, model_name, tokenizer, model)
#             source_texts.append(source_text)
#             translations.append(translation)
#             references.append(reference)
#         except Exception as e:
#             print(f"Error during translation with {model_name}: {str(e)}")
#             continue

#     # Clear GPU memory after processing
#     model.to("cpu")
#     torch.cuda.empty_cache()

#     if translations:
#         metrics = calculate_metrics(references, translations)
#         return translations, metrics
#     return [], {}

# def main():
#     language_pairs = [
#         ("cs-en", "Czech-English"),
#         ("de-en", "German-English"),
#         ("fi-en", "Finnish-English"),
#         ("fr-de", "French-German"),
#         ("gu-en", "Gujarati-English"),
#         ("kk-en", "Kazakh-English"),
#         ("lt-en", "Lithuanian-English"),
#         ("ru-en", "Russian-English"),
#         ("zh-en", "Chinese-English")
#     ]
    
#     for pair_code, pair_name in language_pairs:
#         print(f"\nEvaluating {pair_name} translations...")
#         dataset = load_translation_data(pair_code, num_samples=50)
#         source_lang, target_lang = pair_code.split("-")
        
#         results = {}
#         all_translations = {}
        
#         for model_name in model_names:
#             print(f"Evaluating {model_name}...")
#             translations, metrics = evaluate_model(dataset, source_lang, target_lang, model_name)
#             model_key = model_name.split("/")[-1]  # Use shorter model name for keys
#             results[model_key] = metrics
#             all_translations[model_key] = translations
        
#         # Save results
#         pd.DataFrame(results).to_csv(f"mt_evaluation_results_{pair_code}.csv")
        
#         # Save translations for review
#         translations_df = pd.DataFrame({
#             "Source": [example['translation'][source_lang] for example in dataset],
#             "Reference": [example['translation'][target_lang] for example in dataset],
#             **{f"Translation_{key}": all_translations[key] for key in results.keys()}
#         })
#         translations_df.to_csv(f"translations_comparison_{model}{pair_code}.csv", index=False)
        
#         # Print results
#         print(f"\nResults for {pair_name}:")
#         print(pd.DataFrame(results))

# if __name__ == "__main__":
#     main()